In [19]:
import os
import pandas as pd
import gspread
from google.oauth2 import service_account
from googleapiclient.discovery import build
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [20]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"python-drive-api.json", scope)
client = gspread.authorize(creds)


ValueError: ('Unexpected credentials type', None, 'Expected', 'service_account')

In [21]:
FOLDER_ID = "1FrbwVzp0HvnPeDLtVIpBb-ngFtgsn5uz"
# 1FrbwVzp0HvnPeDLtVIpBb-ngFtgsn5uz
SERVICE_ACCOUNT_FILE = r"C:/Users/Administrator/LIS/GCPL SAMT AI/python-459011-73acbdff4227.json"

In [11]:
OUTPUT_XLSX = "images_metadata.xlsx"
# Column names - change as required
NUMERIC_COLS = ["Num1", "Num2", "Num3"]
# --------------------------------

# Scopes required to read file metadata and create share links
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly",
          "https://www.googleapis.com/auth/drive.readonly"]

In [14]:
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}\n"
                            "If you don't want to use a service account, follow the OAuth flow "
                            "instructions in the comments to generate credentials.json and adapt the auth.")

In [16]:
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('drive', 'v3', credentials=creds, cache_discovery=False)


In [17]:
# Query files in the folder (non-recursive). Adjust pageSize as needed.
query = f"'{FOLDER_ID}' in parents and trashed = false"
files = []
page_token = None
while True:
    response = service.files().list(q=query,
                                    spaces='drive',
                                    fields="nextPageToken, files(id, name, mimeType)",
                                    pageToken=page_token,
                                    pageSize=1000).execute()
    files.extend(response.get('files', []))
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break



HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files?q=%271FrbwVzp0HvnPeDLtVIpBb-ngFtgsn5uz%27+in+parents+and+trashed+%3D+false&spaces=drive&fields=nextPageToken%2C+files%28id%2C+name%2C+mimeType%29&pageSize=1000&alt=json returned "Google Drive API has not been used in project 609771387973 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=609771387973 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'message': 'Google Drive API has not been used in project 609771387973 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=609771387973 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'domain': 'usageLimits', 'reason': 'accessNotConfigured', 'ex

In [ ]:
# Build dataframe
rows = []
for f in files:
    file_id = f.get('id')
    name = f.get('name')
    # label from file name without extension
    label = os.path.splitext(name)[0]
    # public viewing link format — this works for files with appropriate sharing permissions.
    view_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    # direct-download alternative (may require permission): https://drive.google.com/uc?id={file_id}&export=download
    row = {"Image Label": label, "Google Drive Link": view_link}
    for col in NUMERIC_COLS:
        row[col] = ""  # empty numeric fields for you to populate
    rows.append(row)

# If folder is empty, still create headers
df = pd.DataFrame(rows, columns=["Image Label"] + NUMERIC_COLS + ["Google Drive Link"])

df.to_excel(OUTPUT_XLSX, index=False, engine="openpyxl")
print(f"Done — wrote {len(df)} rows to {OUTPUT_XLSX}")
